In [1]:
# import lib
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
import threading
np.random.seed(78)
tf.set_random_seed(678)

In [2]:
# declare
num_epoch    = 10000
ground_truth = np.random.uniform(0,1,10)
expected     = np.zeros(10)
pull_count   = np.zeros(10) 
print(ground_truth)
print(expected)
print(pull_count)

[0.04818123 0.68096301 0.79869607 0.80010859 0.96529725 0.44401982
 0.83123488 0.24656763 0.05027788 0.48743178]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
# train 
for iter in range(num_epoch):
    current_action = np.random.randint(10)
    pull_count[current_action] = pull_count[current_action] + 1
    if np.random.uniform(0,1) < ground_truth[current_action]: current_reward = 1
    else: current_reward = 0
    expected[current_action] = expected[current_action] + (1/pull_count[current_action]) * (current_reward-expected[current_action])
    
print(np.around(ground_truth,3))
print(np.around(expected,3))
print(np.around(pull_count,3))

[0.048 0.681 0.799 0.8   0.965 0.444 0.831 0.247 0.05  0.487]
[0.036 0.673 0.807 0.806 0.969 0.478 0.842 0.24  0.051 0.471]
[1008. 1038.  933.  972. 1023. 1013. 1043. 1006.  965.  999.]


In [4]:
# declare
expected     = np.zeros(10)
pull_count   = np.zeros(10) 
print(ground_truth)
print(expected)
print(pull_count)

def findvalue():
    global expected,pull_count
    for iter in range(num_epoch//2):
        current_action = np.random.randint(10)
        pull_count[current_action] = pull_count[current_action] + 1
        if np.random.uniform(0,1) < ground_truth[current_action]: current_reward = 1
        else: current_reward = 0
        expected[current_action] = expected[current_action] + (1/pull_count[current_action]) * (current_reward-expected[current_action])
  
jobs = []
for i in range(0, 2):
    thread = threading.Thread(target=findvalue())
    jobs.append(thread)

for x in jobs:
    x.start()

for x in jobs:
    x.join()

print(np.around(ground_truth,3))
print(np.around(expected,3))
print(np.around(pull_count,3))

[0.04818123 0.68096301 0.79869607 0.80010859 0.96529725 0.44401982
 0.83123488 0.24656763 0.05027788 0.48743178]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.048 0.681 0.799 0.8   0.965 0.444 0.831 0.247 0.05  0.487]
[0.051 0.682 0.789 0.81  0.977 0.457 0.823 0.231 0.038 0.465]
[1023. 1034. 1027. 1016.  960.  983.  945. 1027. 1019.  966.]


[0.81  0.424 0.829 0.009 0.086 0.463 0.117 0.486 0.44  0.161]
[0.875 0.333 1.    0.    0.    0.5   0.125 0.5   0.2   0.5  ]
[ 8.  3.  1.  4.  6.  4.  8.  6. 10.  2.]


# Reference 
1. Solving the Multi-Armed Bandit Problem – Towards Data Science. (2017). Towards Data Science. Retrieved 24 December 2018, from https://towardsdatascience.com/solving-the-multi-armed-bandit-problem-b72de40db97c
2. 